In [3]:
import torch
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import safetensors

In [4]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-72B-Instruct')
streamer = TextStreamer(tokenizer)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-72B-Instruct', torch_dtype = torch.bfloat16,
    device_map="auto"
)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[2025-04-30 15:55:03,893] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/37 [00:00<?, ?it/s]

In [13]:
state_dict = model.state_dict()

In [11]:
from safetensors import safe_open

folder = 'lora-embedding-128-qwen2.5-72b-malaysian-8k/checkpoint-1700'

f = safe_open(f"{folder}/adapter_model.safetensors", framework="pt", device='cpu')
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [15]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).to(W.device)
    B = f.get_tensor(B).to(W.device)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|██████████████████████████████████████████████████████████████████████████████████████| 562/562 [00:00<00:00, 576.70it/s]


In [ ]:
d = [
    {'role': 'user', 'content': 'butoh hang tu apa dalam bahsa kedah'}
]

inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
generate_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    repetition_penalty=1.05,
    streamer=streamer
)
generation_output = model.generate(**generate_kwargs)

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
butoh hang tu apa dalam bahsa kedah<|im_end|>
<|im_start|>assistant
Ia bermaksud 1. sesuatu yg menutup lubang (pintu dll): ~ 